In [ ]:
# bibliotecas necessárias para o projeto
%pip install selenium
%pip install webdriver-manager

**selenium**: É uma biblioteca que permite a automação de interações com navegadores da web.

**selenium.webdriver.common.by**: Contém constantes para localizar elementos na página da web. Por exemplo, By.XPATH é usado para localizar elementos por XPATH.

**selenium.webdriver.support**: Contém classes e funções relacionadas ao suporte a espera (wait). Ele ajuda a aguardar que certas condições sejam atendidas antes de realizar ações.

**selenium.webdriver.support.ui**: Fornece classes para trabalhar com espera explícita e outras operações relacionadas à interface do usuário.

**webdriver_manager.chrome**: Ajuda a gerenciar os drivers dos navegadores, como o ChromeDriver, tornando mais fácil a configuração e o download do driver correto para a versão do navegador.

**selenium.webdriver.chrome.options**: Contém opções de configuração para o driver do Chrome, como configurar as opções do navegador.

**selenium.webdriver.chrome.service**: Permite configurar e personalizar o serviço do ChromeDriver, que controla o navegador Chrome.

**os**: Fornece funcionalidades relacionadas ao sistema operacional, criar diretórios, mover arquivos, entre outras operações.

**json**: Permite a manipulação de dados no formato JSON, que é comumente usado para armazenar e transmitir dados estruturados.

**re**: Fornece suporte para expressões regulares, que são usadas para fazer correspondência de padrões em strings.

**time**: Este módulo fornece funções para trabalhar com tempo e atrasos no código.

**datetime**: Fornece classes para manipular datas e horas.

**pathlib**: Para manipular caminhos de arquivos e diretórios de forma mais eficiente e segura do que usando strings de caminho.

In [203]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
import os
import json
import re
import time
import datetime
from pathlib import Path

In [212]:
# CONSTANTES
DIRETORIO_DOWNLOAD = os.path.join(Path.home(), "Downloads")
ARQUIVO_CONTROLE = 'Controle\Arquivos.json'
BOTOES = {
    'categoria': 'cphBody_btn',
    'ano': 'cphBody_lkAno',
    'mes': 'cphBody_lkMes',
    'exportar': 'cphBody_ExportarBOLink'
}

In [236]:
def criar_categoria(categoria, expressao, diretorio=None):
    if diretorio is None:
        diretorio = os.path.join(os.getcwd(), os.path.abspath('..\\'), 'Bases', categoria)
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)

    dados = {}
    if os.path.isfile(ARQUIVO_CONTROLE):
        with open(ARQUIVO_CONTROLE, 'r') as controle:
            dados = json.load(controle)

    if categoria not in dados:
        dados[categoria] = {
            "diretorio": diretorio,
            "expressao": expressao,
            "arquivos": {}
        }

    with open(ARQUIVO_CONTROLE, 'w') as controle:
        json.dump(dados, controle, indent=4)

def ordenar_arquivo():
    if os.path.isfile(ARQUIVO_CONTROLE):
        with open(ARQUIVO_CONTROLE, 'r') as controle:
            dados = json.load(controle)

        for categoria in dados:
            for ano in dados[categoria]["arquivos"]:
                dados[categoria]["arquivos"][ano] =\
                    dict(sorted(dados[categoria]["arquivos"][ano].items(),key=lambda item: int(item[0])))
            dados[categoria]["arquivos"] =\
                dict(sorted(dados[categoria]["arquivos"].items(),key=lambda item: int(item[0])))
        dados = dict(sorted(dados.items(), key=lambda item: item[0]))

        with open(ARQUIVO_CONTROLE, 'w') as controle:
            json.dump(dados, controle, indent=4)

def adicionar_arquivo(categoria, arquivo):
    if os.path.isfile(ARQUIVO_CONTROLE):
        with open(ARQUIVO_CONTROLE, 'r') as controle:
            dados = json.load(controle)

        if categoria in dados:
            caminho_arquivo = os.path.join(DIRETORIO_DOWNLOAD, arquivo)
            coincide = re.search(dados[categoria]["expressao"], arquivo)
            if coincide:
                ano = coincide.group(1)
                mes = coincide.group(2)

                if ano not in dados[categoria]["arquivos"]:
                    dados[categoria]["arquivos"][ano] = {}

                if mes not in dados[categoria]["arquivos"][ano]:
                    dados[categoria]["arquivos"][ano][mes] = {
                        "arquivo": arquivo,
                        "tamanho": f'{os.path.getsize(caminho_arquivo) / 1024:_.0f}'.replace(".", ",").replace("_", ".") + ' KB',
                        "data": str(datetime.datetime.fromtimestamp(os.path.getmtime(caminho_arquivo)))
                    }

                with open(ARQUIVO_CONTROLE, 'w') as controle:
                    json.dump(dados, controle, indent=4)

                ordenar_arquivo()

                try:
                    os.rename(caminho_arquivo, os.path.join(dados[categoria]["diretorio"], arquivo))
                    return True
                except Exception as e:
                    print(f"Erro ao mover o arquivo: {arquivo}")
    return False

def adicionar_arquivos_retroativo(diretorio):
    if os.path.isfile(ARQUIVO_CONTROLE):
        with open(ARQUIVO_CONTROLE, 'r') as controle:
            dados = json.load(controle)
        for arquivo in os.listdir(diretorio):
            for categoria in dados:
                coincide = re.search(dados[categoria]["expressao"], arquivo)
                if coincide:
                    ano = coincide.group(1)
                    mes = coincide.group(2)

                    if ano not in dados[categoria]["arquivos"]:
                        dados[categoria]["arquivos"][ano] = {}

                    if mes not in dados[categoria]["arquivos"][ano]:
                        dados[categoria]["arquivos"][ano][mes] = {
                            "arquivo": arquivo,
                            "tamanho": f'{os.path.getsize(os.path.join(diretorio, arquivo)) / 1024:_.0f}'\
                                .replace(".", ",").replace("_", ".") + ' KB',
                            "data": str(datetime.datetime.fromtimestamp(os.path.getmtime(os.path.join(diretorio, arquivo))))
                        }

                    with open(ARQUIVO_CONTROLE, 'w') as controle:
                        json.dump(dados, controle, indent=4)

                    ordenar_arquivo()

                    try:
                        os.rename(os.path.join(diretorio, arquivo), os.path.join(dados[categoria]["diretorio"], arquivo))
                    except Exception as e:
                        print(f"Erro ao mover o arquivo: {arquivo}")

def arquivos_pendentes():
    pendentes = {}
    if os.path.isfile(ARQUIVO_CONTROLE):
        with open(ARQUIVO_CONTROLE, 'r') as controle:
            dados = json.load(controle)
        for categoria in dados:
            # adicionar a expressão regular para a categoria
            pendentes[categoria] = {
                "expressao": dados[categoria]["expressao"],
                "arquivos": {}
            }
            for ano in range(2010, datetime.datetime.now().year + 1):
                pendentes[categoria]["arquivos"][ano] = {}
                if ano == datetime.datetime.now().year:
                    meses = range(1, datetime.datetime.now().month)
                else:
                    meses = range(1, 13)
                meses_pendentes = [mes for mes in meses if str(mes) not in dados[categoria]["arquivos"].get(str(ano), {}).keys()]
                pendentes[categoria]["arquivos"][ano] = meses_pendentes

                if pendentes[categoria]["arquivos"][ano] == []:
                    pendentes[categoria]["arquivos"].pop(ano)
            if pendentes[categoria] == {"expressao": dados[categoria]["expressao"], "arquivos": {}}:
                pendentes.pop(categoria)
    return pendentes



In [237]:
# criar_categoria(categoria='FurtoCelular', expressao='^DadosBO_(\d{4})_(\d{1,2})\(FURTO DE CELULAR\)\.xls$')
# criar_categoria(categoria='RouboCelular', expressao='^DadosBO_(\d{4})_(\d{1,2})\(ROUBO DE CELULAR\)\.xls$')

print(arquivos_pendentes())

{'FurtoCelular': {'expressao': '^DadosBO_(\\d{4})_(\\d{1,2})\\(FURTO DE CELULAR\\)\\.xls$', 'arquivos': {2010: [1], 2011: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2012: [7, 8], 2013: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2018: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2019: [1, 2, 6, 7, 10, 11, 12], 2023: [9, 10]}}, 'RouboCelular': {'expressao': '^DadosBO_(\\d{4})_(\\d{1,2})\\(ROUBO DE CELULAR\\)\\.xls$', 'arquivos': {2010: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2011: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2012: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2013: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2014: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2015: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2016: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2017: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 2018: [1, 2, 3, 4, 5, 6, 7, 8,

In [238]:
def abrir_navegador():
    opcoes = Options()
    # opcoes.add_argument('--headless') # Rodar sem abrir o navegador
    opcoes.add_argument('--auto-open-devtools-for-tabs') # Abrir o console do navegador
    opcoes.add_argument('--start-maximized')
    opcoes.add_experimental_option('prefs', {'download.default_directory': DIRETORIO_DOWNLOAD})
    navegador = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=opcoes)
    navegador.set_page_load_timeout(1200)
    navegador.get('https://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx')
    return navegador

def clicar(botao, valor=''):
    elemento = (By.XPATH, f'//*[@id="{BOTOES[botao]}{valor}"]')
    WebDriverWait(navegador, 30).until(EC.element_to_be_clickable(elemento)).click()


In [239]:
pendentes = arquivos_pendentes()

navegador = abrir_navegador()
print('Acessando site...')
for categoria in pendentes:
    clicar('categoria', categoria)
    for ano in pendentes[categoria]["arquivos"]:
        clicar('ano', ano % 100)
        for mes in pendentes[categoria]["arquivos"][ano]:
            clicar('mes', mes)
            print(f'Exportando {categoria}, {ano}-{mes}...')
            clicar('exportar')

            while True:
                arquivos = os.listdir(DIRETORIO_DOWNLOAD)
                coincide = [arquivo for arquivo in arquivos if re.match(pendentes[categoria]["expressao"], arquivo)]

                if coincide:
                    break
                print('Aguardando arquivo baixar...')
                time.sleep(5)

            for arquivo in coincide:
                sucesso = adicionar_arquivo(categoria, arquivo)
                if sucesso:
                    print(f'Arquivo {arquivo} baixado com sucesso!')
                else:
                    print(f'Erro ao baixar arquivo {arquivo}!')


Acessando site...


TypeError: list indices must be integers or slices, not str